In [ ]:
# Okay so nearly not all imports are being used by this notebook, but this gives a broad overview of the different models/sampling techniques that were tested

# general
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold

# models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, AdaBoostClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from skopt import BayesSearchCV

# evaluation 
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import average_precision_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, balanced_accuracy_score, classification_report, roc_auc_score, RocCurveDisplay, roc_curve, auc, precision_recall_curve, precision_score, PrecisionRecallDisplay

# class imbalance
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler, SMOTENC
from imblearn.ensemble import *
from sklearn.calibration import CalibratedClassifierCV
import relplot as rp
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight

# saving the model
import joblib

In [ ]:
def get_feature_df(df: pd.DataFrame, training:bool=False) -> pd.DataFrame:
    '''
    Returns a dataframe with only the features.
    It also sorts the dataframe in the right format/sorted
    if training is set to true, the target variable will be added
    '''
    feature_cols = ['PATIENTNR', 'GESLACHT', 'LEEFTIJD', 'POSTCODE', 'WOONPLAATS', # patient features
                'AGENDA', 'DESCRIPTION', 'CONSTYPE', 'CODE', 'SPECIALISME','LOCATIE', 'DUUR', # appointment features
                'AfspraakZelfdeDag', 'AFSTAND', 'VerschilInplannenEnAfspraak', 'num_no_shows', 'perc_no_shows', 'stiptheid', 'MaandAfspraak', 'DagAfspraak', 'TijdAfspraak',  # engineerd features
                'num_appointments', 'last_noshow', 'days_since_last_appointment', # engineerd features
                ]
    # if you want to train with the data, add target variable and date
    if training:
        feature_cols.append('STARTDATEPLAN')
        feature_cols.append('no_show')

    return df[feature_cols]

## Loading the data

In [ ]:
FILL_NA = False

pd.set_option('display.max_columns', 400)
df = pd.read_csv('/mnt/data/jmaathuis/no_shows/no_show_all_apps_run2_start_date=2020-01-01_hist=5_improved2.csv', parse_dates=['STARTDATEPLAN'])

In [ ]:
df = get_feature_df(df, training=True)
df = df[~df['DagAfspraak'].isin(['Saturday', 'Sunday'])]  # not predicting appointments in the weekend
df = df[df['CONSTYPE'].isin(['H', 'E', 'V', '*'])]

In [ ]:
df = df[df['STARTDATEPLAN'] >= '2022-01-01']

if FILL_NA:
    df.loc[df['AFSTAND'].isna(), 'AFSTAND'] = df['AFSTAND'].median()
    df.loc[df['num_no_shows'].isna(), 'num_no_shows'] = df['num_no_shows'].mean()
    df.loc[df['perc_no_shows'].isna(), 'perc_no_shows'] = df['perc_no_shows'].mean()
    df.loc[df['stiptheid'].isna(), 'stiptheid'] = df['stiptheid'].mean()
    df.loc[df['DUUR'].isna(), 'DUUR'] = df['DUUR'].mean()
    df.loc[df['TijdAfspraak'].isna(), 'TijdAfspraak'] = df['TijdAfspraak'].value_counts().argmax()
    df.loc[df['days_since_last_appointment'].isna(), 'days_since_last_appointment'] = df['days_since_last_appointment'].mean()


cat_cols = ['GESLACHT', 'WOONPLAATS', 'POSTCODE', 'AGENDA', 'CONSTYPE', 'CODE', 'SPECIALISME', 'LOCATIE', 'DagAfspraak', 'DESCRIPTION', 'AfspraakZelfdeDag']

df[cat_cols] = df[cat_cols].astype('category')
if FILL_NA:
    for col in df.select_dtypes(include='category'):
        df[col] = df[col].cat.codes

## No-Show distributions per specialism

In [ ]:
df_no_shows = pd.DataFrame(columns=['specialisme', 'n_appointments', 'n_no_shows', 'perc_no_shows'])
counts = df['no_show'].value_counts()
df_no_shows.loc[len(df_no_shows.index)] = ['TOTAAL', counts.sum(), counts[1], round(counts[1] / counts.sum() * 100, 2)]


for spec in df['SPECIALISME'].unique():
    counts = df[df['SPECIALISME'] == spec]['no_show'].value_counts()
    if 1 in counts.keys():
        df_no_shows.loc[len(df_no_shows.index)] = [spec, counts.sum(), counts[1], round(counts[1] / counts.sum() * 100, 2)]

df_no_shows.sort_values(by='perc_no_shows')

In [ ]:
# remvoing noisy specialisms (due to different reasons)
spec_to_remove = ['RAD', 'APO', 'ONC', 'GEV', 'ORT', 'GGZ', 'PSY', 'ANE']
df = df[~df['SPECIALISME'].isin(spec_to_remove)]

## Train-test split

The train test split is based on year:   
* The training set contains data from 2021 to 2023.   
* Thest test set contains data from the years 2024

In [ ]:
df_all = df.copy()

df = df[df['STARTDATEPLAN'] >= pd.Timestamp("2020-01-01 00:00:00")]
df_test = df[df['STARTDATEPLAN'] > pd.Timestamp("2024-01-01 00:00:00")]

df = df[df['STARTDATEPLAN'] < pd.Timestamp("2024-01-01 00:00:00")]
df = df.drop(columns=['STARTDATEPLAN'])
df_test = df_test.drop(columns=['STARTDATEPLAN', 'PATIENTNR'])

X, y = df.drop(columns=['no_show', 'PATIENTNR']), df['no_show']
X_test, y_test = df_test.drop(columns=['no_show']), df_test['no_show']

In [ ]:
df_all['no_show'].value_counts(dropna=False)
df_all[df_all['STARTDATEPLAN'] > pd.Timestamp("2024-01-01 00:00:00")]['no_show'].value_counts()

## Modeling

In [ ]:
class XGBClassifierResampled(XGBClassifier):

    def fit(self, X, y, sample_weight=None):
        resampler = RandomOverSampler(sampling_strategy=0.4)
        X, y = resampler.fit_resample(X, y)
        super().fit(X, y, sample_weight=class_weight.compute_sample_weight('balanced',  y=y))

In [ ]:
# for now this works fine, later run a grid/bayesian search
classifier_params = {
    'device':'cuda', 
    'enable_categorical':True,
    'learning_rate':0.15, 
    'max_cat_to_onehot':25,
    'max_cat_threshold':5,
    'reg_alpha':10,
    'scale_pos_weight':0.5,
    'n_estimators': 350,
    'importance_type': 'gain'
}

clf = XGBClassifier(**classifier_params)
stratified_k_fold = StratifiedKFold(n_splits=3, shuffle=True)
metrics = ('precision', 'recall', 'roc_auc', 'f1', 'average_precision', 'balanced_accuracy')

scores = cross_validate(clf, X, y, 
                        cv=stratified_k_fold, 
                        scoring=metrics, 
                        return_estimator=True,
                        fit_params={'sample_weight': class_weight.compute_sample_weight('balanced',  y=y)}
                        )

for metric in ['f1', 'roc_auc', 'average_precision', 'balanced_accuracy']:
    metric = 'test_' + metric   
    print(f'{metric}: {scores[metric].mean():.3f} (+/-{np.std(scores[metric]):.3f})')

### Precision recall plot

In [ ]:
# doing another single run to show the confusion matrix and PR-AUC
X_train = X
y_train = y

clf = XGBClassifier(**classifier_params)

clf.fit(X_train, y_train, sample_weight=class_weight.compute_sample_weight('balanced',  y=y_train))
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]
# y_pred = (y_pred_proba > 0.6).astype(int)
print(classification_report(y_test, y_pred, digits=3))
print(roc_auc_score(y_test, y_pred_proba))
print(average_precision_score(y_test, y_pred_proba))

cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Show', 'No-Show'])
disp.plot(cmap='Blues')
plt.show()

PrecisionRecallDisplay.from_predictions(y_test, y_pred_proba)
plt.ylim(0, 0.7)
plt.grid(alpha=.2)
plt.xlim(0, 0.6)
plt.show()

### Performance per Specialism

In [ ]:
X_plot = X_test.copy()

y_pred_proba = clf.predict_proba(X_plot)[:,1]
X_plot['y_score'] = y_pred_proba
X_plot['y_true'] = y_test

specs = []
pr_aucs = []
roc_aucs = []
for spec in X_plot.SPECIALISME.unique():
    X_plot_spec = X_plot[X_plot['SPECIALISME'] == spec]
    if len(X_plot_spec) > 2000:
        try:
            pr_aucs.append(average_precision_score(X_plot_spec['y_true'], X_plot_spec['y_score']))
            roc_aucs.append(roc_auc_score(X_plot_spec['y_true'], X_plot_spec['y_score']))
            specs.append(spec)
        except:
            continue

plt.title('PR-AUC')
plt.barh(specs, pr_aucs)
plt.show()

plt.title('ROC-AUC')
plt.barh(specs, roc_aucs)
plt.show()

### Grid search

In [ ]:
param_space = {
    'enable_categorical': [True],
    'max_cat_to_onehot': (1, 50, 'uniform'),
    'max_cat_threshold': (1, 10, 'uniform'),
    'scale_pos_weight': (0, 1),
    'reg_alpha': (0, 20, 'uniform'),
    'reg_lambda': (0, 20, 'uniform'),
    'n_estimators': (0, 1000, 'uniform'),
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'grow_policy': ['depthwise', 'lossguide'],
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1.0, 'uniform'),
    'colsample_bytree': (0.5, 1.0, 'uniform'),
    'colsample_bylevel': (0.5, 1.0, 'uniform'),
    'colsample_bynode': (0.5, 1.0, 'uniform'),
    'gamma': (0.01, 10.0, 'log-uniform'),
}


# Initialize the Bayesian Optimization optimizer
opt = BayesSearchCV(
    estimator=XGBClassifier(),
    search_spaces=param_space,
    scoring='average_precision',  
    n_iter=100,  
    cv=5,  
)

# Run Bayesian Optimization
opt.fit(X_train, y_train)

# Retrieve the best hyperparameters
best_params = opt.best_params_
best_score = opt.best_score_

# Train the final model with the best hyperparameters
final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# Evaluate the final model on a separate test dataset
final_model_score = roc_auc_score(y_test, final_model.predict_proba(X_test)[:,1])

### Feature importances

In [ ]:
clf = final_model

feature_imps = {feature: importance for feature, importance in zip(X.columns, clf.feature_importances_)}
feature_imps = dict(sorted(feature_imps.items(), key=lambda x: x[1], reverse=True))

plt.figure(figsize=(10,4))
plt.bar(feature_imps.keys(), feature_imps.values(), color='C1', alpha=.5)
plt.xticks(rotation=90)
plt.show()

### Saving the model

In [ ]:
# do a full retrain on the whole dataset (train+test) so that we can use this for inference
df_all = df_all.sample(frac=1)  # shuffles the dataset
df_all = df_all.drop(columns=['STARTDATEPLAN', 'PATIENTNR'], axis=1)
X, y = df_all.drop(columns=['no_show']), df_all['no_show']
clf = XGBClassifier(**classifier_params)

clf.fit(X, y, sample_weight=class_weight.compute_sample_weight('balanced',  y=y))


joblib.dump(clf, '../5_Deployment/no_show_model_v2.joblib')